In [2]:
import pandas as pd
import gc

In [3]:
sunat_train = pd.read_csv('../data/sunat_train.csv')
y_train = pd.read_csv('../data/y_train.csv', index_col = 'key_value')
sunat_test = pd.read_csv('../data/sunat_test.csv')

In [4]:
############################# sunat_train
### consta de 1462036 registros con 17 features
### fecalta y fecbaja tienen valores nulos de 1.5% y 49.5%
### el 34% de key_value no existe en sunat_train

### tipcontribuyente Tiene 50 clases ### freq 29 
### tippersona Tiene 3 clases ### Freq 1 
### ciiu ,codigo de actividad economica, Tiene 285 clases ### Freq 285
### ubigeo Tiene 1655 clases ### Freq 442
### condiciondomicilio tiene 16 clases ### Freq 4
### estadocontribuyente tiene 13 clases ### Freq 3
### codvia tiene 29 clases ### Freq 0
### codzona tiene 36 clases ### Freq 0
### contabilidad tiene 12 clases ### Freq 6
### facturacion tiene 21 clases ### Freq 11
### domiciliado tiene 3 clases ### Freq 1
### comercioexterior tiene 14 clases ### Freq 2
### cargorele tiene 656 clases ### Freq 635
### codentidadtributo tiene 9 clases ### Freq 6
### estadotributo tiene 3 clases ### Freq 1


In [5]:
### hay algunos clientes que tienen fecha de alta, pero no fecha de baja, siendo estos 723714 registros
### los clientes que no tienen fecha de alta, tampoco tienen fecha de baja, por obvias razones. Aunque has un caso donde
#un cliente no tiene fecha de alta pero si fecha de baja siendo esto cerca de 20897 registros
### clientes que tienen fecha de alta , asi como fecha de baja son cerca de 738321 registros

### cuando se trata de un cliente nuevo en el rcc, la proporcion de valores nulos de la fecha de alta se mantiene, 
# mientras que la fecha de baja aumenta de 49% as 86%
### no necesariamente cleintes que tienen un solo registro en la sunat_train, van a tener un solo registro en el rcc_train
### El 18% de cliente tiene de fecha alta mayor a la fecha de baja --- Es raro

In [6]:
### tippersona Tiene 3 clases ### Freq 1 
### domiciliado tiene 3 clases ### Freq 1
### codentidadtributo tiene 9 clases ### Freq 6
### estadotributo tiene 3 clases ### Freq 1

In [7]:
dict_ = dict(zip(list(sunat_train.dtypes[sunat_train.dtypes == int].index), ['int32']*len(sunat_train.dtypes[sunat_train.dtypes == int].index)))
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [8]:
sunat_train['fecdiff'] = sunat_train['fecbaja'].fillna(max(sunat_train['fecbaja'])) - sunat_train['fecalta']
sunat_test['fecdiff'] = sunat_test['fecbaja'].fillna(max(sunat_test['fecbaja'])) - sunat_test['fecalta']

In [9]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=-1
    return moda
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['sum'],
           'fecbaja':['sum'],
           'fecdiff':['sum']
          }

In [10]:
sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [11]:
sunat_train_ = pd.concat([sunat_train_, y_train.loc[sunat_train_.index]], axis=1)

tippersona_mode_me = sunat_train_.groupby(['tippersona_mode'])['target'].mean().to_dict() 
domiciliado_mode_me = sunat_train_.groupby(['domiciliado_mode'])['target'].mean().to_dict() 
codentidadtributo_mode_me = sunat_train_.groupby(['codentidadtributo_mode'])['target'].mean().to_dict() 
estadotributo_mode_me = sunat_train_.groupby(['estadotributo_mode'])['target'].mean().to_dict() 

sunat_train_['tippersona_mode_me'] =  sunat_train_['domiciliado_mode'].map(tippersona_mode_me) 
sunat_train_['domiciliado_mode_me'] =  sunat_train_['tippersona_mode'].map(domiciliado_mode_me) 
sunat_train_['codentidadtributo_mode_me'] =  sunat_train_['codentidadtributo_mode'].map(codentidadtributo_mode_me) 
sunat_train_['estadotributo_mode_me'] =  sunat_train_['estadotributo_mode'].map(estadotributo_mode_me) 

sunat_test_['tippersona_mode_me'] =  sunat_test_['domiciliado_mode'].map(tippersona_mode_me) 
sunat_test_['domiciliado_mode_me'] =  sunat_test_['tippersona_mode'].map(domiciliado_mode_me) 
sunat_test_['codentidadtributo_mode_me'] =  sunat_test_['codentidadtributo_mode'].map(codentidadtributo_mode_me) 
sunat_test_['estadotributo_mode_me'] =  sunat_test_['estadotributo_mode'].map(estadotributo_mode_me) 

In [12]:
sunat_train_.drop(['domiciliado_mode','tippersona_mode','codentidadtributo_mode','estadotributo_mode', 'target'], axis=1, inplace=True)
sunat_test_.drop(['domiciliado_mode','tippersona_mode','codentidadtributo_mode','estadotributo_mode'], axis=1, inplace=True)

In [16]:
#Función para obtener variables con alta correlación
def get_correlated(data, cut=0.85):
    correlated_features = set()
    correlation_matrix = data.corr()
    
    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > cut:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
            
    return correlated_features
print(sunat_train_.shape)
correlated = get_correlated(sunat_train_,cut=0.9)
sunat_train_ = sunat_train_.drop(columns=correlated)
sunat_test_ = sunat_test_.drop(columns=correlated)
print(sunat_train_.shape)

(233866, 33)
(233866, 30)


In [23]:
sunat_train_

,tipcontribuyente_nunique,tipcontribuyente_mode,tippersona_nunique,ciiu_nunique,ciiu_mode,ubigeo_nunique,ubigeo_mode,condiciondomicilio_nunique,condiciondomicilio_mode,estadocontribuyente_nunique,...,comercioexterior_mode,cargorele_mode,codentidadtributo_nunique,estadotributo_nunique,fecalta_sum,fecbaja_sum,fecdiff_sum,tippersona_mode_me,codentidadtributo_mode_me,estadotributo_mode_me
key_value,,,,,,,,,,,,,,,,,,,,,
0,1,9,1,1,178,1,404,1,0,1,...,2,635,1,1,0.971584,0.000000,0.00000,0.150869,0.146867,0.146918
1,1,0,1,1,285,1,1670,1,0,1,...,2,635,1,1,0.566027,0.000000,0.00000,0.150869,0.146867,0.146918
2,1,0,1,1,285,1,228,1,0,1,...,1,635,1,1,0.701213,0.000000,0.00000,0.150869,0.146867,0.146918
3,1,9,1,1,177,1,1761,1,0,1,...,2,635,1,1,4.536984,0.000000,0.00000,0.150869,0.159896,0.145160
5,1,9,1,1,283,2,-1,1,0,2,...,2,635,1,1,-1.706851,-2.556051,-2.04059,0.150869,0.146867,0.146918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358481,1,9,1,1,178,1,442,1,0,1,...,1,635,1,1,1.917886,0.000000,0.00000,0.150869,0.146867,0.146918
358482,1,0,1,1,253,1,557,1,0,1,...,1,635,1,1,0.430841,0.000000,0.00000,0.150869,0.146867,0.146918
358484,1,0,1,1,285,1,13,1,0,1,...,1,635,1,1,1.782700,0.000000,0.00000,0.150869,0.159896,0.145160


In [21]:
sunat_train_.to_csv('../data/intermediate/sunat_train.csv')
sunat_test_.to_csv('../data/intermediate/sunat_test.csv')